In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import cv2

Using plaidml.keras.backend backend.


In [2]:
def get_pickles():
    combined_dict = pickle.load(open('../Pickles/TTSDict.p', 'rb'))
    
    x_train = combined_dict['train']['images']
    x_test = combined_dict['test']['images']
    
    y_train = combined_dict['train']['labels']
    y_test = combined_dict['test']['labels']
    
    validation = combined_dict['validation']
    
    return x_train, x_test, y_train, y_test, validation 

x_train, x_test, y_train, y_test, validation  = get_pickles() 

In [3]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(10000, 250, 250, 3) (992, 250, 250, 3)
(10000,) (992,)


In [4]:
def get_mobilenet(dim):
    model = Sequential()
    optimizer = Adam(lr = .0005)
    baseModel = MobileNetV2(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim))
    
    model.add(baseModel)
    model.add(AveragePooling2D(pool_size=(7, 7)))
    model.add(Flatten(name="flatten"))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation="sigmoid", name = 'Output'))

    
    for layer in baseModel.layers:
        layer.trainable = False
        
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/Mobilenet1.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 32
    

dim = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
mobilenet = get_mobilenet(dim =dim)

    
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
augmentation.fit(x_train)
mobilenet_history = mobilenet.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"
F:\ProgramFiles\conda\envs\FaceMaskEmotionDetection\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Epoch 1/2000
162/313 [==============>...............] - ETA: 1:19 - loss: 0.1684 - acc: 0.9321

INFO:plaidml:Analyzing Ops: 1808 of 2049 operations complete


313/313 [==============================] - 175s 559ms/step - loss: 0.1360 - acc: 0.9472 - val_loss: 0.2561 - val_acc: 0.8931

Epoch 00001: val_loss improved from inf to 0.25612, saving model to ModelWeights/Mobilenet1.h5
Epoch 2/2000
313/313 [==============================] - 136s 435ms/step - loss: 0.1010 - acc: 0.9657 - val_loss: 0.1712 - val_acc: 0.9194

Epoch 00002: val_loss improved from 0.25612 to 0.17125, saving model to ModelWeights/Mobilenet1.h5
Epoch 3/2000
313/313 [==============================] - 149s 476ms/step - loss: 0.0850 - acc: 0.9684 - val_loss: 0.1228 - val_acc: 0.9395

Epoch 00003: val_loss improved from 0.17125 to 0.12283, saving model to ModelWeights/Mobilenet1.h5
Epoch 4/2000
313/313 [==============================] - 140s 448ms/step - loss: 0.0859 - acc: 0.9681 - val_loss: 0.1080 - val_acc: 0.9516

Epoch 00004: val_loss improved from 0.12283 to 0.10800, saving model to ModelWeights/Mobilenet1.h5
Epoch 5/2000
313/313 [==============================] - 138s 442m